In [1]:
import plotly.graph_objs as go
import pandas as pd
import requests
import math
import numpy as np
from plotly.grid_objs import Grid, Column
import plotly.plotly as py

In [2]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [3]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [4]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [37]:
num_indicators = 32
cmdf = complete_measurements_df
ind_list = [f'ind{i+1}' for i in range(num_indicators)]

selected_category = 'B'
cmdf = complete_measurements_df
filtered_df = cmdf[cmdf.category == selected_category]
epsa_codes = list(filtered_df.code.unique())
years = list(filtered_df.sort_values('year').year.unique())

def get_data_rating(epsa_code, year):
    values = list(cmdf[(cmdf.year == year) & (cmdf.code == epsa_code)][ind_list].iloc[0])
    num_reported = sum([0 if math.isnan(val) else 1 for val in values])
    return num_reported / num_indicators * 100

In [42]:
colors = [
    '#1f77b4', '#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

colors = colors * 3

columns = [
    Column(epsa_codes, 'epsa_codes'),
    Column(years, 'years'),
    Column(colors[1:], 'colors')
]

for year in years:
    column_name1 = f'{year}_x'
    column_data1 =  [year] * len(epsa_codes)
    column1 = Column(column_data1, column_name1)
    
    column_name2 = f'{year}_y'
    column_data2 = [get_data_rating(epsa_code, year) for epsa_code in epsa_codes]
    column2 = Column(column_data2, column_name2)
    
    columns += [column1, column2]
    
for epsa_code in epsa_codes:
    
    column_name3 = f'{epsa_code}_y'
    column_data3 = [get_data_rating(epsa_code, year) for year in years]
    column3 = Column(column_data3, column_name3)
    
    columns.append(column3)
    
grid = Grid(columns)
py.grid_ops.upload(grid, 'data_ranking_2017')

'https://plot.ly/~sergiochumacero/90/'

In [43]:
data = [go.Scatter(
    xsrc= grid.get_column_reference(f'2014_x'),
    ysrc= grid.get_column_reference(f'2014_y'),
    textsrc= grid.get_column_reference('epsa_codes'),
    mode='markers+text',
    showlegend=False,
)]

for epsa_code in epsa_codes:
    line_trace = go.Scatter(
        xsrc= grid.get_column_reference('years'),
        ysrc= grid.get_column_reference(f'{epsa_code}_y'),
        mode='lines',
        name=epsa_code,
    )
    data.append(line_trace)

In [44]:
animation_settings = dict(
    frame = dict(duration=1200, redraw=False),
    fromcurrent = False,
    transition = dict(duration=1200, easing='cubic-in-out'),
)

layout = dict(
    xaxis= dict(
        title='Años',
        range=[2013.8, 2017.2],
        autorange= False,
        dtick=1,
    ),
    yaxis= dict(
        title='Porcentaje de variables reportadas (%)',
#         dtick=1,
#         range=[50,100],
#         autorange=False,
    ),
    title= f'Ranking EPSAS - Porcentaje de variables reportadas<br>Categoría {selected_category}',
    updatemenus= [dict(
        x=0,
        y=0,
        type= 'buttons',
        buttons=[dict(
            label='Animar',
            method='animate',
            args= [None],
        )],
    )],
    hovermode='closest',
#     updatemenus = [dict(
#         buttons= [dict(
#             args= [[str(y) for y in years], animation_settings],
#             label= 'Animar',
#             method= 'animate',
#         )],
#         direction= 'left',
#         pad= dict(r=10, t=87),
#         showactive= False,
#         type= 'buttons',
#         x= 0.1,
#         y= 0,
#         xanchor= 'right',
#         yanchor= 'top',
#     )]
)

In [46]:
frames = [{'data': [go.Scatter(
    xsrc=grid.get_column_reference(f'{year}_x'),
    ysrc=grid.get_column_reference(f'{year}_y'),
    textsrc=grid.get_column_reference('epsa_codes'),
    textposition='top center',
    mode='markers',
    marker=dict(
        symbol='circle',
        size=20,
        colorsrc=grid.get_column_reference('colors')),
)]} for year in years]

fig = go.Figure(data=data, layout=layout, frames=frames)

In [47]:
py.icreate_animations(fig, filename='data_rankings_2017')